<div dir="rtl">
تیم ۲۰ بازیابی:
نگین جعفری،
محمدمهدی جراحی،
امین قاسم‌زاده
</div>

In [1]:
# !pip install pandas, tqdm
import json
import tse_entity_matcher as tem
import tqdm
# This takes a little time. Because it compiles many regular expressions.

100%|██████████| 620/620 [00:04<00:00, 128.61it/s]
16it [00:00, 594.49it/s]


<div dir="rtl">
کدهای اصلی در فایل پایتون قرار داده شده‌اند و در آنجا نیز توضیح مختصری درباره‌ی نحوه‌ی عملکرد داده‌ایم. در این نوت‌بوک صرفا اجرای کدها رو برخی نمونه‌ها را خواهیم دید.
</div>

<div dir="rtl">
لیست نمادها و اسامی بورسی در فایل‌های bourseview_symbols_details.json و rahavard_symbols_details.json موجود هستند. این داده‌ها کرال شده‌اند.
</div>

In [2]:
tem.bourseview_symbols_details['items'][0]

{'isin': 'IRO3Z0290001',
 'symbolPouya': 'Z0291',
 'symbolPouyaFa': 'نفت قشم',
 'namePouya': 'Gheshm Oil Co.',
 'namePouyaFa': 'سرمایه گذاری نفت قشم',
 'type': 'CommonStock',
 'tseTickerKey': '15082904528024302',
 'numberOfOutstandingShares': 30294000,
 'marketCap': 302940000000,
 'freeFloat': None,
 'tickerTypeKey': 1,
 'tickerKey': 74720,
 'company_type_key': None,
 'exchange': {'isin': 'IRIFBNO',
  'symbolLocal': 'IFB',
  'symbolPouya': 'IRIFB',
  'namePouya': 'Iran Fara Bourse',
  'tradingState': 'Close'},
 'indices': {'indexOverallPouya': {'isin': 'IREXOVERALLINDEX',
   'exchange': {'isin': 'IREX'},
   'namePouya': 'Iran Exchange'}},
 'company': None,
 'industry': {'industryName': 'Unknown',
  'isin': 'Unknown',
  'sectorKey': 0,
  'sectorName': 'Unknown',
  'industryFaName': 'نامشخص',
  'sectorFaName': 'نامشخص',
  'tickers': []}}

<div dir="rtl">
همچنین لیست وقایع بورسی در events.xlsx است. این وقایع قابلیت پشتبانی از regex را دارند. همچنین در صورتی که نیاز به وجود عدد است، می‌توانید از #NUM استفاده کنید. در ادامه به کمک توابع parsi.io در نظر گرفته خواهد شد.
</div>
<div dir="rtl">
برای ادامه‌ی توسعه‌ی ماژول نوشته شده، لازم است لیست اصطلاحات در فایل اکسل به مرور تکمیل شود.
</div>

In [3]:
tem.events_df

,رجکس اصلاح,نوع اصطلاح
0,(#NUM )؟(درصد |واحد )؟(ضرر|سود),سود/ضرر
1,(اطلاعیه (ی )؟|آگهی |اعلامیه )؟((افشا (ی )؟(اط...,اطلاعیه
2,(حقیقی|حقوقی),شخصیت
3,(#NUM درصد |#NUM واحد )؟افزایش سرمایه,افزایش سرمایه
4,تقسیم سود,تقسیم سود
5,دامنه (ی )؟نوسان,دامنه نوسان
6,نوسان( شدید)؟,نوسان
7,سهم (رانتی|خوب|بد|آشغال),صفت سهم
8,(#NUM )؟(درصد |واحد )؟(تاثیر )؟(مثبت|منفی),رشد سهم
9,گزارش( فعالیت)؟ (ماهانه|فصلی|سالانه|#NUM ماهه),گزارش


<div dir="rtl">
همچنین لیست اسامی بی‌ارزش (مثلا فولاد مبارکه اصفهان با فولاد مبارکه تفاوت ندارد. پس اصفهان جزو کلمات بی‌ارزش است.) در junk_words.xlsx است.
</div>
<div dir="rtl">
برای ادامه‌ی توسعه‌ی ماژول نوشته شده، لازم است لیست لغات بی‌ارزش در فایل اکسل به مرور تکمیل شود.
</div>

In [4]:
tem.junk_words_df

0    اصفهان
1       نفت
Name: لیست بخش‌های تکراری اسامی, dtype: object

In [5]:
def run(text: str):
    dict_list = tem.find_entities(text)
    print(json.dumps(dict_list, indent=4, ensure_ascii=False).encode('utf8').decode())





<div dir="rtl">
در فایل پایتون tse_entity_matcher.py ما داده‌ها را می‌خوانیم و عبارات منظم بسیار متنوعی می‌سازیم. مثلا نیم‌فاصله‌ها ها را در نظر می‌گیریم. ی نکره را در نظر می‌گیریم. وجود یا عدم وجود کلمات junk را در نظر می‌گیریم. اعداد را شناسایی می‌کنیم. در صورتی که یکی از اسپن‌های کشف شده زیر مجموعه‌ی دیگری باشد، اسپن کوچکتر را نادیده می‌گیریم.
</div>
<div dir="rtl">
یکی از مزایای بسیار بزرگ کد ما، در نظر گرفتن کلمات junk است. مثلا فولاد مبارکه اصفهان با فولاد مبارکه تفاوت ندارد.
</div>
<div dir="rtl">
یکی دیگر از مزایای بزرگ کد ما، در نظر گرفتن نمادهای حق‌تقدم می‌باشد.
</div>
<div dir="rtl">
<b>
از آنجا که کرال کردن و تمیز کردن‌های اولیه مانند تبدیل حروف عربی به فارسی، جزو اهداف تمرین ۲ می‌باشد، ما در این تمرین فرض را بر این می‌گذاریم که داده‌ها کرال شده و برخی تمیزکاری‌های بسیار مقدماتی مانند حذف علائم نگارشی و حذف کسره/فتحه/تشدید/... و تبدیل حروف عربی به فارسی، حذف کشـــــیدگی‌ها، از قبل انجام شده‌است. همچنین فرض بر این است که در متن، علامت ~ موجود نمی‌باشد و در صورت وجود حذف شده‌است. زیرا از این علامت خاص در کدهایمان استفاده‌ی خاصی کرده‌ایم.
</b>
</div>
<div dir="rtl">
توضیحات تکمیلی درباره‌ی نحوه‌ی عملکرد ماژول نوشته شده، در فایل پایتون در کنار کدها قرار دارد. در این نوت‌بوک صرفا اجرای کدها رو برخی نمونه‌ها را خواهیم دید.
</div>


test_find_1
<div dir="rtl">
تشخیص نام دقیق شرکت‌ها
</div>

In [6]:
text = "فولاد مبارکه اصفهان"
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "فولاد",
        "marker": "فولاد مبارکه اصفهان",
        "span": [
            0,
            19
        ]
    }
]


test_find_2
<div dir="rtl">
تشخیص ی نکره در نام شرکت
</div>

In [7]:
text = "فولاد مبارکه‌ی اصفهان"
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "فولاد",
        "marker": "فولاد مبارکه‌ی اصفهان",
        "span": [
            0,
            21
        ]
    }
]


test_find_3
<div dir="rtl">
تشخیص نمادها مانند فرابورس و همچنین سایر اصلاحات حاوی اعداد
</div>

In [8]:
text = 'جریان آغاز معاملات فزر با ۱۵.۲ واحد تاثیر مثبت بر روند صعودی بازار فرابورس اثر گذار بود.'
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "فزر",
        "marker": "فزر",
        "span": [
            19,
            22
        ]
    },
    {
        "type": "رشد سهم",
        "marker": "۱۵.۲ واحد تاثیر مثبت",
        "span": [
            26,
            46
        ]
    },
    {
        "type": "روند",
        "marker": "روند صعودی",
        "span": [
            50,
            60
        ]
    },
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "فرابورس",
        "marker": "فرابورس",
        "span": [
            67,
            74
        ]
    }
]


test_find_4

In [9]:
text = 'برکت همین افشای ب باعث شد سهم سه درصد مثبت شود. بخاطر همین میگم پیگیر باشید.'
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "برکت",
        "marker": "برکت",
        "span": [
            0,
            4
        ]
    },
    {
        "type": "اطلاعیه",
        "marker": "افشای ب",
        "span": [
            10,
            17
        ]
    },
    {
        "type": "رشد سهم",
        "marker": "سه درصد مثبت",
        "span": [
            30,
            42
        ]
    }
]


test_find_5

In [10]:
text = 'گزارش فعالیت ماهانه دوره ۱ ماهه منتهی به ۱۴۰۰̸۰۹̸۳۰ برای دیران منتشر شد.'
run(text)

[
    {
        "type": "گزارش",
        "marker": "گزارش فعالیت ماهانه",
        "span": [
            0,
            19
        ]
    },
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "دیران",
        "marker": "دیران",
        "span": [
            57,
            62
        ]
    }
]


test_find_6
<div dir="rtl">
تشخیص نام انگلیسی
</div>

In [11]:
text = 'Sina Insurance is a good Insurance company!'
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "وسین",
        "marker": "Sina Insurance",
        "span": [
            0,
            14
        ]
    }
]


test_find_7
<div dir="rtl">
تشخیص آیدی سهم
</div>

In [12]:
text = ' IRO7VSNP0001 is a good Insurance company!'
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "وسین",
        "marker": "IRO7VSNP0001",
        "span": [
            1,
            13
        ]
    }
]


test_find_8
<div dir="rtl">
تشخیص فرم‌های مختلف سهم انرژی۳
</div>

In [13]:
text = 'انرژی ۳  یا انرژی‌۳ یا انرژی۳ یا انرژی 3 یا انرژی‌3 یا انرژی3 خیلی خوب است.'
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "انرژی3",
        "marker": "انرژی ۳",
        "span": [
            0,
            7
        ]
    },
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "انرژی3",
        "marker": "انرژی‌۳",
        "span": [
            12,
            19
        ]
    },
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "انرژی3",
        "marker": "انرژی۳",
        "span": [
            23,
            29
        ]
    },
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "انرژی3",
        "marker": "انرژی 3",
        "span": [
            33,
            40
        ]
    },
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "انرژی3",
        "marker": "انرژی‌3",
        "span": [
            44,
            51
        ]
    },
    {
        "type": "نماد",
        "type_de

test_find_9

In [14]:
text = "سهم بازگشایی شد"
run(text)

[
    {
        "type": "بازگشایی",
        "marker": "بازگشایی",
        "span": [
            4,
            12
        ]
    }
]


test_find_10

In [15]:
text = "سهم افزایش سرمایه خورد"
run(text)

[
    {
        "type": "افزایش سرمایه",
        "marker": "افزایش سرمایه",
        "span": [
            4,
            17
        ]
    }
]


test_find_11

In [16]:
text = "به لحظات 12.30 و صف خرید نزدیک می شویم"
run(text)

[
    {
        "type": "صف",
        "marker": "صف خرید",
        "span": [
            17,
            24
        ]
    }
]


test_find_12

In [17]:
text = "به لحظات 12.30 و روندی صعودی نزدیک می شویم"
run(text)

[
    {
        "type": "روند",
        "marker": "روندی صعودی",
        "span": [
            17,
            28
        ]
    }
]


test_find_13
<div dir="rtl">
تشخیص اصطلاحات تکنیکالی
</div>

In [18]:
text = 'یک نکته  تکنیکالی هم در صورت دستکاری نشدن اضافه کنم؛ کندلی که روز سه شنبه گذشته ثبت کرد بلحاظ فنی چون دو سوم آن از ابر خارج شده و کندل کاملی است و می تواند بعنوان تایید و خروج از ابر درنظر گرفت. و پولبک آن هم چهارشنبه توضیحاتش گذشت. '
run(text)

[
    {
        "type": "تکنیکال",
        "marker": "کندلی",
        "span": [
            53,
            58
        ]
    },
    {
        "type": "تکنیکال",
        "marker": "کندل",
        "span": [
            130,
            134
        ]
    },
    {
        "type": "تکنیکال",
        "marker": "پولبک",
        "span": [
            197,
            202
        ]
    }
]


test_find_14

In [19]:
text = "فراکابی ها همگی خوبند. فراکاب ها و علی الخصوص بورس و کالا کندل پرقدرتی زدن"
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "فراکاب",
        "marker": "فراکابی",
        "span": [
            0,
            7
        ]
    },
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "فراکاب",
        "marker": "فراکاب",
        "span": [
            23,
            29
        ]
    },
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "بورس",
        "marker": "بورس",
        "span": [
            46,
            50
        ]
    },
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "کالا",
        "marker": "کالا",
        "span": [
            53,
            57
        ]
    },
    {
        "type": "تکنیکال",
        "marker": "کندل",
        "span": [
            58,
            62
        ]
    }
]


test_find_15

In [20]:
text = 'ریزش شدید جفت ارز های فارکس '
run(text)

[
    {
        "type": "ریزش",
        "marker": "ریزش",
        "span": [
            0,
            4
        ]
    }
]


test_find_16

In [21]:
text = 'تشکیل کف دوقلو و پیش بسوی ۱۲۰۰'
run(text)

[
    {
        "type": "تکنیکال",
        "marker": "کف",
        "span": [
            6,
            8
        ]
    },
    {
        "type": "تکنیکال",
        "marker": "دوقلو",
        "span": [
            9,
            14
        ]
    }
]


test_find_17

In [22]:
text = 'گزارش نه ماهه هم که خوب نبوده'
run(text)

[
    {
        "type": "گزارش",
        "marker": "گزارش نه ماهه",
        "span": [
            0,
            13
        ]
    }
]


test_find_18

In [23]:
text = 'در یک نگاه کلاسیک الگوی سروشانه رو هم ساخته'
run(text)

[
    {
        "type": "تکنیکال",
        "marker": "سروشانه",
        "span": [
            24,
            31
        ]
    }
]


test_find_19

In [24]:
text = 'مکدی هم شرایط خوبی داره و کراس رو به بالا زده'
run(text)

[
    {
        "type": "تکنیکال",
        "marker": "مکدی",
        "span": [
            0,
            4
        ]
    },
    {
        "type": "تکنیکال",
        "marker": "کراس",
        "span": [
            26,
            30
        ]
    }
]


test_find_20

In [25]:
text = ' بازار با عبور از ابر کومو و کف سازی دلاری'
run(text)

[
    {
        "type": "تکنیکال",
        "marker": "ابر کومو",
        "span": [
            18,
            26
        ]
    },
    {
        "type": "تکنیکال",
        "marker": "کف سازی",
        "span": [
            29,
            36
        ]
    }
]


test_find_21

In [26]:
text = 'یه کد به کد داشت تو قیمت ۱۰۰۰'
run(text)

[
    {
        "type": "تکنیکال",
        "marker": "کد به کد",
        "span": [
            3,
            11
        ]
    }
]


test_find_22

In [27]:
text = ' پس از توقف نماد معاملاتی باز خواهد شد.'
run(text)

[
    {
        "type": "توقف نماد",
        "marker": "توقف نماد",
        "span": [
            7,
            16
        ]
    }
]


test_find_23

In [28]:
text = 'خبر دارم از #انرژی ۲'
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "انرژی2",
        "marker": "انرژی ۲",
        "span": [
            13,
            20
        ]
    }
]


test_find_24
<div dir="rtl">
توانایی در نظر گرفتن کلمات بی‌ارزش مانند اصفهان در شرکت فولاد مبارکه‌ی اصفهان
</div>

In [29]:
text = "سهم فولاد مبارکه بسیار عالی است و سهم فولاد مبارکه‌ی شهر اصفهان عالی است."
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "فولاد",
        "marker": "فولاد مبارکه",
        "span": [
            4,
            16
        ]
    },
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "فولاد",
        "marker": "فولاد مبارکه‌ی",
        "span": [
            38,
            52
        ]
    }
]


test_find_25
<div dir="rtl">
توانایی تطبیق «پالایش نفت بندر عباس» با «پالایش نفت» ذ
</div>

In [30]:
text = 'پالایش بندرعباس دارم ببین اصن عالیه'
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "شبندر",
        "marker": "پالایش بندرعباس",
        "span": [
            0,
            15
        ]
    }
]


test_find_26

In [31]:
text = 'نفت قشم ولک عالیه'
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "نفت قشم",
        "marker": "نفت قشم",
        "span": [
            0,
            7
        ]
    }
]


<h2 dir="rtl">
تشخیص نمادهای حق‌تقدم
</h2>
test_find_hagh_1

In [32]:
text = "من کویرح را فروختم و به کویر تبدیل کردم."
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد حق تقدم شرکت",
        "symbol": "کویر",
        "marker": "کویرح",
        "span": [
            3,
            8
        ]
    },
    {
        "type": "نماد",
        "type_detailed": "نماد شرکت",
        "symbol": "کویر",
        "marker": "کویر",
        "span": [
            24,
            28
        ]
    }
]


test_find_hagh_2

In [33]:
text = "من سهم حق تقدم کویر را خریدم"
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد حق تقدم شرکت",
        "symbol": "کویر",
        "marker": "حق تقدم کویر",
        "span": [
            7,
            19
        ]
    }
]


test_find_hagh_3

In [34]:
text = "من سهم حق‌تقدم کویر را خریدم"
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد حق تقدم شرکت",
        "symbol": "کویر",
        "marker": "حق‌تقدم کویر",
        "span": [
            7,
            19
        ]
    }
]


test_find_hagh_4

In [35]:
text = "من سهم تقدم کویر را خریدم"
run(text)

[
    {
        "type": "نماد",
        "type_detailed": "نماد حق تقدم شرکت",
        "symbol": "کویر",
        "marker": "تقدم کویر",
        "span": [
            7,
            16
        ]
    }
]


<h2 dir="rtl">
بررسی سرعت عملکرد کد روی دیتای بزرگ
</h2>

In [36]:
%%time
# Test Speed on 500KB text.
import pandas as pd

comments = pd.read_csv("comments.csv")
for comment in tqdm.tqdm(comments['body']):
    tem.find_entities(comment)

100%|██████████| 3541/3541 [02:21<00:00, 25.05it/s]

CPU times: user 2min 20s, sys: 1.22 s, total: 2min 21s
Wall time: 2min 21s
